# IBM WatsonX + Exa Integration

This notebook demonstrates how to combine IBM WatsonX AI with Exa's web search capabilities to create an AI assistant that can search and use current information.

## Getting API Keys

### IBM WatsonX API Key:
1. Go to [IBM Cloud](https://cloud.ibm.com)
2. Create/Login to account
3. Navigate to WatsonX.AI
4. Create a project
5. Get Project ID from project settings
6. Create API key from IAM settings
7. Get URL based on your region

### Exa API Key:
1. Visit [Exa Dashboard](https://dashboard.exa.ai)
2. Create/Login to account
3. Go to API section
4. Generate new API key
5. Set usage limits
6. Copy key immediately (shown only once)

## 1. Setup

First, let's install the required packages:

In [ ]:
!pip install ibm-watsonx-ai exa-py python-dotenv

In [ ]:
import os
import logging
from dotenv import load_dotenv

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Load from .env file
load_dotenv()

# Or set directly (replace with your keys)
os.environ['IBM_WATSONX_API_KEY'] = 'your_ibm_key'
os.environ['IBM_WATSONX_URL'] = 'https://us-south.ml.cloud.ibm.com'
os.environ['IBM_WATSONX_PROJECT_ID'] = 'your_project_id'
os.environ['EXA_API_KEY'] = 'your_exa_key'

In [ ]:
from ibm_watsonx_ai import APIClient, Credentials

def get_client():
    """Create and return a configured IBM WatsonX AI client."""
    try:
        credentials = Credentials(
            url=os.getenv('IBM_WATSONX_URL'),
            api_key=os.getenv('IBM_WATSONX_API_KEY')
        )
        return APIClient(
            credentials=credentials,
            project_id=os.getenv('IBM_WATSONX_PROJECT_ID')
        )
    except Exception as e:
        logger.error(f"Error creating client: {str(e)}")
        raise

In [ ]:
from typing import List, Dict, Any
from exa_py import Exa

class ExaSearchClient:
    def __init__(self, api_key: str):
        self.client = Exa(api_key=api_key)
    
    def search(self, query: str, num_results: int = 3) -> List[Dict[str, Any]]:
        try:
            response = self.client.search_and_contents(
                query,
                type="auto",
                num_results=num_results,
                livecrawl="always",
                text=True
            )
            
            formatted_results = []
            if isinstance(response, dict) and 'data' in response:
                for result in response['data'].get('results', []):
                    if 'text' in result:
                        formatted_results.append({
                            "title": result.get('title', 'No title'),
                            "text": result['text'][:500] + "..." if len(result['text']) > 500 else result['text'],
                            "url": result.get('url', ''),
                            "score": result.get('score', 0.0),
                            "published_date": result.get('published_date', '')
                        })
            
            return formatted_results
        except Exception as e:
            logger.error(f"Search error: {str(e)}")
            return []

In [ ]:
import json

def create_web_search_tool():
    return {
        "type": "function",
        "function": {
            "name": "web_search",
            "description": "Search the web for current information. Always cite sources and include dates when available.",
            "parameters": {
                "type": "object",
                "properties": {
                    "query": {
                        "type": "string",
                        "description": "The search query"
                    }
                },
                "required": ["query"]
            }
        }
    }

class ToolExecutor:
    def __init__(self, exa_client):
        self.exa_client = exa_client
    
    def execute_tool(self, tool_call):
        try:
            if tool_call["type"] != "function":
                return "Unsupported tool type"
            
            function_call = tool_call["function"]
            if function_call["name"] == "web_search":
                query = json.loads(function_call["arguments"])["query"]
                results = self.exa_client.search(query)
                
                return "\n\n".join(
                    f"Source: {r['title']}\n" \
                    f"URL: {r['url']}\n" \
                    f"Date: {r.get('published_date', 'Not available')}\n" \
                    f"Content: {r['text']}\n"
                    for r in results
                )
            return "Unknown tool"
        except Exception as e:
            logger.error(f"Tool execution error: {str(e)}")
            return f"Error executing tool: {str(e)}"

In [ ]:
from ibm_watsonx_ai.foundation_models import ModelInference

class WatsonXModel:
    def __init__(self, exa_api_key, model_params=None):
        self.client_wrapper = get_client()
        self.exa_client = ExaSearchClient(exa_api_key)
        self.tool_executor = ToolExecutor(self.exa_client)
        
        # Default model parameters
        self.model_params = {
            'temperature': 0.7,
            'max_new_tokens': 1024,
            'min_new_tokens': 1,
            'repetition_penalty': 1.1
        }
        if model_params:
            self.model_params.update(model_params)
        
        self.model = ModelInference(
            model_id="mistralai/mistral-large",
            credentials=self.client_wrapper.credentials,
            project_id=os.getenv('IBM_WATSONX_PROJECT_ID')
        )
        self.model.params = self.model_params
    
    def ask(self, question: str) -> str:
        try:
            messages = [{"role": "user", "content": question}]
            tools = [create_web_search_tool()]
            
            # Get initial response with tool calls
            response = self.model.chat(messages=messages, tools=tools)
            
            # Handle tool calls if any
            if "choices" in response and response["choices"]:
                choice = response["choices"][0]
                if "message" in choice and "tool_calls" in choice["message"]:
                    tool_calls = choice["message"]["tool_calls"]
                    
                    # Execute tools and get results
                    tool_results = []
                    for tool_call in tool_calls:
                        result = self.tool_executor.execute_tool(tool_call)
                        tool_results.append(result)
                    
                    # Add results to conversation
                    messages.extend([
                        {
                            "role": "assistant",
                            "content": None,
                            "tool_calls": tool_calls
                        },
                        {
                            "role": "tool",
                            "content": "\n\n".join(tool_results),
                            "tool_call_id": tool_calls[0]["id"]
                        }
                    ])
                    
                    # Get final response
                    final_response = self.model.chat(messages=messages)
                    if "choices" in final_response and final_response["choices"]:
                        return final_response["choices"][0]["message"]["content"].strip()
                elif "content" in choice["message"]:
                    # Return direct response if no tool calls needed
                    return choice["message"]["content"].strip()
            
            return "No response generated"
        except Exception as e:
            logger.error(f"Error in ask method: {str(e)}")
            return f"An error occurred: {str(e)}"

In [ ]:
# Initialize the model
model = WatsonXModel(exa_api_key=os.getenv('EXA_API_KEY'))

# Try some questions
questions = [
    "What are the latest developments in AI regulation in 2024?",
    "What are recent breakthroughs in quantum computing?",
    "What are the current trends in renewable energy?"
]

for question in questions:
    print(f"\nQuestion: {question}")
    print("\nAnswer:")
    print(model.ask(question))
    print("\n" + "-"*80)